# 6 - Markets, Firms, and Economic Activity in CXB

Originally modified from the snapping notebook, some holdover code is likely

This notebook prepares data for modeling potential accessibility to destinations in CXB via gravity models

In [1]:
import os, sys
import time
import pandas as pd
import geopandas as gpd
import numpy as np

import networkx as nx
from scipy import spatial
from functools import partial
import pyproj

from shapely.ops import transform
from shapely import wkt
from shapely.geometry import Point, MultiPoint
from shapely.wkt import loads

import ipyparallel as ipp

import GOSTnets as gn
import geopy

In [2]:
#!/usr/bin/env python
import psutil
# gives a single float value
psutil.cpu_percent()
# gives an object with many fields
psutil.virtual_memory()
# you can convert that object to a dictionary 
dict(psutil.virtual_memory()._asdict())

{'total': 34342551552,
 'available': 26375348224,
 'percent': 23.2,
 'used': 7967203328,
 'free': 26375348224}

Optional enable all columns -- this will negatively affect performance

In [3]:
# pd.set_option('display.max_columns', None)

Set path names and file names

In [4]:
input_pth = r'inputs\\dests'
interm_pth = r'intermediate\\'
dest_pth = r'final'
res_pth = r'results'
res_date = '200521'

tab_pth = r'../../../Tabular'
geo_pth = r'../../../GEO'
origin_folder = r'..\..\..\GEO\Population\CXB'

administrative boundaries

In [6]:
# load admin spatial data

adm2 = gpd.read_file(os.path.join(geo_pth,'Boundaries/adm2/bgd_admbnda_adm2_bbs_20180410.shp'))
adm2.crs = 'epsg:4326'

# adm3 = gpd.read_file(os.path.join(geo_pth,'Boundaries/bangladesh_upazila_boundary/bangladesh_upazila_boundary.shp')) # 8 in CXB

adm3 = gpd.read_file(os.path.join(geo_pth,'Boundaries/adm3/bgd_admbnda_adm3_bbs_20180410.shp'))
adm3.crs = 'epsg:4326'

adm4 = gpd.read_file(os.path.join(geo_pth,'Boundaries/adm4/bgd_admbnda_adm4_bbs_20180410.shp')) # 75 in CXB
adm4.crs = 'epsg:4326'

In [7]:
# Fix types for later joining
adm3['ADM3_PCODE'] = adm3['ADM3_PCODE'].astype(str)
adm4['ADM4_PCODE'] = adm4['ADM4_PCODE'].astype(str)

In [8]:
adm3['ADM3_EN'] = adm3['ADM3_EN'].str.title()

population / origins

In [9]:
origin_file = r'hrsl_2018_cxb_pts.shp'

destinations

In [10]:
cxb_fil = r'cxb_ctr.shp'
chitt_fil = r'chittagong.shp'
health_fil = r'hc_merge_200324_4326.shp'
primary_fil = r'school_category_primary.gpkg'
secondary_fil = r'school_category_secondary.gpkg'
tertiary_fil = r'school_category_tertiary.gpkg'
matar_fil = r'martarbari.shp'
mkts_fil = r'mkts_merge_4326.shp'
gc_fil = r'cxb_lged_gc_moved_4326.shp'

markets and growth centers (major markets) are our main destinations of interest in this notebook

In [11]:
allm = gpd.read_file(os.path.join(input_pth,mkts_fil))
gcs = gpd.read_file(os.path.join(input_pth,gc_fil))

### Load in markets, join in EC data

In [12]:
firms = pd.read_stata(os.path.join(tab_pth,'EconCensus/EC_2013_union_level_v3_cxb.dta')).rename(columns={'adm4_pcode' : 'ADM4_PCODE','adm3_pcode' : 'ADM3_PCODE'})

firms.drop(columns=['shape_leng','shape_area','adm4_en','adm4_ref','adm4alt1en','adm4alt2en','adm3_en','ADM3_PCODE','adm2_en','adm2_pcode','adm1_en','adm1_pcode','adm0_en','adm0_pcode','date'],axis=1,inplace=True)

In [13]:
jobs = pd.read_stata(os.path.join(tab_pth,'EconCensus/EC_2013_union_level_v4_cxb.dta')).rename(columns={'adm4_pcode' : 'ADM4_PCODE','adm3_pcode' : 'ADM3_PCODE'})

jobs.drop(columns=['shape_leng','shape_area','adm4_en','adm4_ref','adm4alt1en','adm4alt2en','adm3_en','ADM3_PCODE','adm2_en','adm2_pcode','adm1_en','adm1_pcode','adm0_en','adm0_pcode','date'],axis=1,inplace=True)

jobs = jobs[['ADM4_PCODE','good_jobs_tot_ser','bad_jobs_tot_ser','good_jobs_tot_','bad_jobs_tot_','good_jobs_tot_ind','bad_jobs_tot_ind','good_jobs_fsize_4','bad_jobs_fsize_4','good_jobs_fsize_5','bad_jobs_fsize_5']]

In [14]:
list(jobs)

['ADM4_PCODE',
 'good_jobs_tot_ser',
 'bad_jobs_tot_ser',
 'good_jobs_tot_',
 'bad_jobs_tot_',
 'good_jobs_tot_ind',
 'bad_jobs_tot_ind',
 'good_jobs_fsize_4',
 'bad_jobs_fsize_4',
 'good_jobs_fsize_5',
 'bad_jobs_fsize_5']

In [15]:
firms['ADM4_PCODE'] = firms['ADM4_PCODE'].astype(str)
jobs['ADM4_PCODE'] = jobs['ADM4_PCODE'].astype(str)

In [16]:
firms.head(2)

,ADM4_PCODE,female_tot_workers_tot_,male_tot_workers_tot_,total_workers_tot_,number_firms_tot_,female_tot_workers_tot_ind,male_tot_workers_tot_ind,total_workers_tot_ind,number_firms_tot_ind,female_tot_workers_tot_ser,...,total_workers_5_oser,number_firms_5_oser,female_tot_workers_5_tran,male_tot_workers_5_tran,total_workers_5_tran,number_firms_5_tran,female_tot_workers_5_whol,male_tot_workers_5_whol,total_workers_5_whol,number_firms_5_whol
0,20221610,138.0,4873.0,5011.0,1100,62.0,4281.0,4343.0,739,76.0,...,52.0,4,0.0,0.0,0.0,0,0.0,11.0,11.0,1
1,20221612,33.0,395.0,428.0,236,18.0,269.0,287.0,125,15.0,...,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0


In [17]:
jobs.head(2)

,ADM4_PCODE,good_jobs_tot_ser,bad_jobs_tot_ser,good_jobs_tot_,bad_jobs_tot_,good_jobs_tot_ind,bad_jobs_tot_ind,good_jobs_fsize_4,bad_jobs_fsize_4,good_jobs_fsize_5,bad_jobs_fsize_5
0,20221610,652.0,16.0,4925.0,86.0,4273.0,70.0,3560.0,42.0,238.0,20.0
1,20221612,141.0,0.0,427.0,1.0,286.0,1.0,11.0,0.0,38.0,0.0


In [18]:
adm4.head(2)

,Shape_Leng,Shape_Area,ADM4_EN,ADM4_PCODE,ADM4_REF,ADM4ALT1EN,ADM4ALT2EN,ADM3_EN,ADM3_PCODE,ADM2_EN,...,ADM1_EN,ADM1_PCODE,ADM0_EN,ADM0_PCODE,date,validOn,validTo,hrsl18sum,wp20sum,geometry
0,0.286382,0.001944,Abaipur,40448010,None,None,None,Shailkupa,404480,Jhenaidah,...,Khulna,40,Bangladesh,BD,2015-01-01,2018-04-10,None,34843.005475,19666.171850,"POLYGON ((89.29995 23.62161, 89.30005 23.62142..."
1,0.098597,0.000317,Abdullahpur,30599411,None,None,None,Tongibari,305994,Munshiganj,...,Dhaka,30,Bangladesh,BD,2015-01-01,2018-04-10,None,14234.900010,20241.963911,"POLYGON ((90.48251 23.55370, 90.48244 23.55351..."


Merge admin data with firm and jobs data

In [19]:
adm4 = pd.merge(adm4,firms,how='left',on='ADM4_PCODE')
adm4 = pd.merge(adm4,jobs,how='left',on='ADM4_PCODE')

In [20]:
adm4 = adm4[adm4['male_tot_workers_5_whol'].isna() == False]
len(adm4) # should be 75

75

In [21]:
adm4.head(2)

,Shape_Leng,Shape_Area,ADM4_EN,ADM4_PCODE,ADM4_REF,ADM4ALT1EN,ADM4ALT2EN,ADM3_EN,ADM3_PCODE,ADM2_EN,...,good_jobs_tot_ser,bad_jobs_tot_ser,good_jobs_tot_,bad_jobs_tot_,good_jobs_tot_ind,bad_jobs_tot_ind,good_jobs_fsize_4,bad_jobs_fsize_4,good_jobs_fsize_5,bad_jobs_fsize_5
73,0.216665,0.001496,Ali Akbar Deil,20224513,None,None,None,Kutubdia,202245,Cox's Bazar,...,135.0,4.0,1499.0,7.0,1364.0,3.0,789.0,4.0,14.0,0.0
255,0.257282,0.001881,Badarkhali,20221610,None,None,None,Chakaria,202216,Cox's Bazar,...,652.0,16.0,4925.0,86.0,4273.0,70.0,3560.0,42.0,238.0,20.0


join in the admin data to the points

In [22]:
allm = gpd.sjoin(allm,adm4.reset_index(),how='left',op='within')
gcs = gpd.sjoin(gcs,adm4.reset_index(),how='left',op='within')

In [23]:
gcs.head(2)

,FID_,AREA,PERIMETER,TYPE,NAME_WS,ID_WS,Upa_name,geometry,index_right,index,...,good_jobs_tot_ser,bad_jobs_tot_ser,good_jobs_tot_,bad_jobs_tot_,good_jobs_tot_ind,bad_jobs_tot_ind,good_jobs_fsize_4,bad_jobs_fsize_4,good_jobs_fsize_5,bad_jobs_fsize_5
0,0.0,0.0,0.0,70.0,Khutakhali Bazar,42216047,Chakoria,POINT (92.06685 21.61506),42,2756,...,1267.0,24.0,4272.0,56.0,3005.0,32.0,1410.0,16.0,100.0,16.0
1,0.0,0.0,0.0,70.0,Manikpur Bazar,42216049,Chakoria,POINT (92.14787 21.75228),68,4614,...,332.0,2.0,1206.0,11.0,874.0,9.0,78.0,10.0,36.0,0.0


In [24]:
allm_adms = pd.DataFrame(allm.groupby('ADM4_PCODE')['NAME_WS'].apply(list)).rename(columns={'NAME_WS': 'All_mkts_names'})
gc_adms = pd.DataFrame(gcs.groupby('ADM4_PCODE')['NAME_WS'].apply(list)).rename(columns={'NAME_WS': 'Growth_centers_names'})

In [25]:
allm_adms['All_mkts_names'] = allm_adms['All_mkts_names'].astype(str).str.strip('[]')
gc_adms['Growth_centers_names'] = gc_adms['Growth_centers_names'].astype(str).str.strip('[]')

In [26]:
adm4 = pd.merge(adm4,allm_adms,how='left',on='ADM4_PCODE')
adm4 = pd.merge(adm4,gc_adms,how='left',on='ADM4_PCODE')

Export

In [27]:
adm4.to_file("results/spatial/adm4_firms.geojson",driver='GeoJSON')
adm4.to_file("results/spatial/adm4_firms.shp")
adm4.to_csv("results/tables/adm4_firms.csv")

## Load in OD Matrics, assign NN accessibility scores to individual markets

find the market for each origin node

In [28]:
current_allm_odm = pd.read_csv(os.path.join(res_pth,res_date,'OD_matrix_current_NN_All_markets.csv'))
current_gc_odm = pd.read_csv(os.path.join(res_pth,res_date,'OD_matrix_current_NN_Growth_centers.csv'))

In [29]:
current_allm_odm['mkt_node'] = current_allm_odm.iloc[:,1:].idxmin(axis=1)
current_gc_odm['mkt_node'] = current_gc_odm.iloc[:,1:].idxmin(axis=1)

In [30]:
current_allm_odm = current_allm_odm[['Unnamed: 0','mkt_node']].rename(columns={'Unnamed: 0' : 'O_ID'})
current_gc_odm = current_gc_odm[['Unnamed: 0','mkt_node']].rename(columns={'Unnamed: 0' : 'O_ID'})

In [31]:
current_gc_odm.columns

Index(['O_ID', 'mkt_node'], dtype='object')

In [32]:
current_gc_odm.head(2)

,O_ID,mkt_node
0,8192,9677
1,1,10053


load in origin nodes

In [34]:
current_allmkts = r'final_cxb_current_od_grid_All_markets.csv'
current_growthcenters = r'final_cxb_current_od_grid_Growth_centers.csv'

In [35]:
allm_od = pd.read_csv(os.path.join(res_pth,res_date,current_allmkts))
gc_od = pd.read_csv(os.path.join(res_pth,res_date,current_growthcenters))

In [36]:
gc_od.head(2)

,O_ID,Unnamed: 0,VALUE,geometry,O_ID.1,walk_to_road_net,on_network_time,total_time_net,walk_time_direct,PLOT_TIME_SECS,PLOT_TIME_MINS,choice
0,1916,0,20.187097,POINT (91.8750000000112 21.91583333332886),1916,369.883438,679.087746,1048.971183,3514.279368,1048.971183,17.482853,net
1,2149,1,20.187097,POINT (91.86333333334434 21.91555555555108),2149,25.753736,601.330535,627.084271,3231.074261,627.084271,10.451405,net


In [37]:
len(allm_od)

106074

In [38]:
allm_od = pd.merge(allm_od,current_allm_odm,how='left',on='O_ID')
gc_od = pd.merge(gc_od,current_gc_odm,how='left',on='O_ID')

In [39]:
len(gc_od)

106074

In [40]:
len(list(set(allm_od['mkt_node'])))

150

In [41]:
import numpy as np

allm_od['mkt_pop'] = allm_od.groupby('mkt_node')['PLOT_TIME_MINS'].transform(np.sum) # mkt_pop not aggregating correctly
gc_od['mkt_pop'] = gc_od.groupby('mkt_node')['PLOT_TIME_MINS'].transform(np.sum)

allm_od['access_time_wtd_allm'] = (allm_od['PLOT_TIME_MINS'] * (allm_od['VALUE'] / allm_od['mkt_pop']))
gc_od['access_time_wtd_gcs'] = (gc_od['PLOT_TIME_MINS'] * (gc_od['VALUE'] / gc_od['mkt_pop']))

In [42]:
np.sum(allm_od[allm_od['mkt_node'] == '7777']['access_time_wtd_allm'])

38.08367437

In [43]:
allm_access = allm_od.groupby('mkt_node').agg(access_time_allm = ('access_time_wtd_allm',np.sum))
gc_access = gc_od.groupby('mkt_node').agg(access_time_gc = ('access_time_wtd_gcs',np.sum),    ) 

### Loading in NN for each market

In [44]:
allm_snap = pd.read_csv(os.path.join(res_pth,res_date,'All_markets_current_snapped.csv'))
gc_snap = pd.read_csv(os.path.join(res_pth,res_date,'Growth_centers_current_snapped.csv'))

In [45]:
allm_snap['NN'] = allm_snap['NN'].astype(str)
gc_snap['NN'] = gc_snap['NN'].astype(str)

In [48]:
allm_snap.head(2)

,Unnamed: 0,FID_,AREA,PERIMETER,TYPE,NAME_WS,ID_WS,Upa_name,SB_CODE,layer,path,geometry,NN,NN_dist
0,0,0.0,0.0,0.0,70.0,Khutakhali Bazar,42216047,Chakoria,NaN,cxb_lged_gc_moved_4326,C:/Users/wb467985/Documents/Data/Bangladesh/CX...,POINT (92.0668513 21.6150622),2733,0.000000
1,1,0.0,0.0,0.0,70.0,Manikpur Bazar,42216049,Chakoria,NaN,cxb_lged_gc_moved_4326,C:/Users/wb467985/Documents/Data/Bangladesh/CX...,POINT (92.14787184505889 21.75228261453107),10444,65.287869


In [49]:
allm_access.head(2)

,access_time_allm
mkt_node,
10020,30.783904
10023,35.005145


In [50]:
allm_snap = pd.merge(allm_snap,allm_access,how='left',left_on='NN',right_on='mkt_node')
gc_snap = pd.merge(gc_snap,gc_access,how='left',left_on='NN',right_on='mkt_node')

In [51]:
allm_snap['ID_WS'] = allm_snap['ID_WS'].astype(int).astype(str)
gc_snap['ID_WS'] = gc_snap['ID_WS'].astype(int).astype(str)
allm['ID_WS'] = allm['ID_WS'].astype(int).astype(str)
gcs['ID_WS'] = gcs['ID_WS'].astype(int).astype(str)

In [52]:
allm_snap = allm_snap.drop_duplicates(subset='ID_WS',keep='first')
gc_snap = gc_snap.drop_duplicates(subset='ID_WS',keep='first')

join the files

In [58]:
allm = pd.merge(allm,allm_snap[['NAME_WS','ID_WS','NN','access_time_allm']],how='left',on=['NAME_WS','ID_WS'])
gcs = pd.merge(gcs,gc_snap[['NAME_WS','ID_WS','NN','access_time_gc']],how='left',on=['NAME_WS','ID_WS'])

In [59]:
allm = allm.drop_duplicates(subset='ID_WS',keep='first')
gcs = gcs.drop_duplicates(subset='ID_WS',keep='first')

Export

In [61]:
# tabular

# allm.to_csv('results/tables/all_markets_access.csv')
# gcs.to_csv('results/tables/growth_centers_access.csv')

In [62]:
# spatial

allm.to_file('results/spatial/all_markets_complete.geojson',driver="GeoJSON")
gcs.to_file('results/spatial/growth_centers_complete.geojson',driver="GeoJSON")

# allm.to_file('results/spatial/markets_and_firms.gpkg',layer='all_markets',driver="GPKG")
# gcs.to_file('results/spatial/markets_and_firms.gpkg',layer='growth_centers',driver="GPKG")

## Gravity modeling

#### Scale markets by population

Load in and join data. We ended up only using growth centers but this could easily be adapted to "all markets"

The "current" and "ua" labels denote data according to "current" and "upgrade all" transportation investment scenarios

In [108]:
# allm = gpd.read_file('results/spatial/all_markets_complete.geojson')
gcs = gpd.read_file('results/spatial/growth_centers_complete.geojson')
adm4 = gpd.read_file("results/spatial/adm4_firms.shp")

In [109]:
# am_pop = gpd.read_file(os.path.join(geo_pth,'Economic/CXB_Markets.gpkg'),layer='all_cxb_markets_32646')
gc_pop = gpd.read_file(os.path.join(geo_pth,'Economic/CXB_Markets.gpkg'),layer='all_cxb_GCs_32646')

In [110]:
# am_pop.ID_WS = am_pop.ID_WS.astype(int).astype(str)
gc_pop.ID_WS = gc_pop.ID_WS.astype(int).astype(str)

In [111]:
# allm = pd.merge(allm,am_pop[['ID_WS','1km_pop','500m_pop']],how='left',on='ID_WS')
gcs = pd.merge(gcs,gc_pop[['ID_WS','1km_pop','500m_pop']],how='left',on='ID_WS')

Check IDs for use in gravity modeling

In [112]:
# pd.set_option('max_columns',None)

In [113]:
gcs.head(2)

,FID_,AREA,PERIMETER,TYPE,NAME_WS,ID_WS,Upa_name,index_right,index,Shape_Leng,...,bad_jobs_tot_ind,good_jobs_fsize_4,bad_jobs_fsize_4,good_jobs_fsize_5,bad_jobs_fsize_5,NN,access_time_gc,geometry,1km_pop,500m_pop
0,0.0,0.0,0.0,70.0,Khutakhali Bazar,42216047,Chakoria,42,2756,0.586368,...,32.0,1410.0,16.0,100.0,16.0,2733,28.429473,POINT (92.06685 21.61506),8632,3988
1,0.0,0.0,0.0,70.0,Manikpur Bazar,42216049,Chakoria,68,4614,0.291419,...,9.0,78.0,10.0,36.0,0.0,10444,24.981090,POINT (92.14787 21.75228),2928,1716


##### Prepare OD files

In [114]:
current_gc_odm = pd.read_csv(os.path.join(res_pth,res_date,'OD_matrix_current_NN_Growth_centers.csv'))
ua_gc_odm = pd.read_csv(os.path.join(res_pth,res_date,'OD_matrix_upgrade_all_NN_Growth_centers.csv'))

In [115]:
current_orig_gc = pd.read_csv(os.path.join(res_pth,res_date,'final_cxb_current_od_grid_Growth_centers.csv'))
ua_orig_gc = pd.read_csv(os.path.join(res_pth,res_date,'final_cxb_upgrade_all_od_grid_Growth_centers.csv'))

Load in origin files, group population figures by origin node for weighting

In [116]:
# current_orig_gc.O_ID.value_counts()

In [117]:
current_orig_gc.head(2)

,O_ID,Unnamed: 0,VALUE,geometry,O_ID.1,walk_to_road_net,on_network_time,total_time_net,walk_time_direct,PLOT_TIME_SECS,PLOT_TIME_MINS,choice
0,1916,0,20.187097,POINT (91.8750000000112 21.91583333332886),1916,369.883438,679.087746,1048.971183,3514.279368,1048.971183,17.482853,net
1,2149,1,20.187097,POINT (91.86333333334434 21.91555555555108),2149,25.753736,601.330535,627.084271,3231.074261,627.084271,10.451405,net


Summarize origin populations by origin node, to enable appropriate population weighting

In [118]:
cur_orig_grp = pd.pivot_table(current_orig_gc,values='VALUE',index='O_ID',aggfunc='sum').rename(columns={'VALUE' : 'Pop'}).reset_index()
ua_orig_grp = pd.pivot_table(ua_orig_gc,values='VALUE',index='O_ID',aggfunc='sum').rename(columns={'VALUE' : 'Pop'}).reset_index()

In [119]:
cur_orig_grp

,O_ID,Pop
0,1,31.045279
1,4,114.251023
2,5,217.316955
3,7,132.310104
4,8,211.696166
...,...,...
4854,10564,246.591100
4855,10565,343.466175
4856,10566,141.309677
4857,10571,201.919993


##### Put the origins and destinations in alphanumeric order by node so that they line up and process correctly within the gravity model

In [120]:
# function for sorting alphanumerically

import re

def sorted_nicely( l ): 
    """ Sort the given iterable in the way that humans expect.""" 
    convert = lambda text: int(text) if text.isdigit() else text 
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(l, key = alphanum_key)

In [121]:
# sort by O_IDs, then dest node IDs

current_gc_odm = current_gc_odm.sort_values('Unnamed: 0').reindex(sorted_nicely(current_gc_odm.columns), axis=1)
ua_gc_odm = ua_gc_odm.sort_values('Unnamed: 0').reindex(sorted_nicely(ua_gc_odm.columns), axis=1)

# reset O_ID column to the front
current_gc_odm = current_gc_odm[ ['Unnamed: 0'] + [ col for col in current_gc_odm.columns if col != 'Unnamed: 0' ] ]
ua_gc_odm = ua_gc_odm[ ['Unnamed: 0'] + [ col for col in ua_gc_odm.columns if col != 'Unnamed: 0' ] ]

# set the Dest_ID column back to index so the shape is the same as the dWeight shape
current_gc_odm.set_index('Unnamed: 0',inplace=True)
ua_gc_odm.set_index('Unnamed: 0',inplace=True)

In [122]:
current_gc_odm

,2733,3280,3308,3883,5181,5280,5911,6315,6462,7000,...,10135,10200,10224,10241,10256,10328,10444,10453,10560,10565
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
1,2272.300389,2186.708575,5827.317668,6249.088465,9294.650641,1117.720698,1792.546491,654.146921,6144.428620,5462.178512,...,852.318141,1604.182003,3223.906975,3610.553109,3886.534188,2618.145058,5763.307033,3358.367410,8511.363741,6218.706146
4,6021.506079,3094.212779,8489.007040,9649.593283,13043.856331,4866.926388,5541.752181,4567.196816,9893.634310,9211.384202,...,4765.368035,5517.231898,7136.956870,7523.603003,7799.584082,6367.350748,9512.512723,7107.573100,12260.569431,9967.911836
5,1763.970750,2947.451699,5564.426244,5740.758826,8786.321002,609.391059,1284.216853,1261.782415,5636.098981,4953.848873,...,1606.538956,2195.556781,3978.127790,4364.773924,4640.755003,2109.815420,5254.977395,2850.037771,8003.034102,5710.376507
7,4175.820728,1248.527429,6387.503549,7548.089791,11198.170980,3021.241037,3696.066831,2721.511465,8047.948959,7365.698851,...,2919.682684,3671.546547,5291.271519,5677.917653,5953.898731,4521.665398,7666.827373,5261.887749,10414.884080,8122.226485
8,3759.240518,1499.447769,5914.788022,7075.374265,10781.590771,2604.660828,3279.486621,2304.931256,7631.368750,6949.118642,...,2503.102475,3254.966338,4874.691310,5261.337443,5537.318522,4105.085188,7250.247163,4845.307539,9998.303871,7705.646275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10564,3334.002642,7230.287790,8686.490052,6540.649534,3688.347610,4488.582333,3813.756539,5544.618506,3654.585239,910.624762,...,5460.115292,4708.251429,5561.168233,5174.522099,4898.541020,2988.157972,4692.508499,2287.568875,2905.060710,612.403115
10565,3946.405757,7842.690905,9298.893167,7153.052649,3075.944495,5100.985448,4426.159654,6157.021621,4266.988354,1523.027877,...,6072.518407,5320.654545,6173.571348,5786.925214,5510.944135,3600.561087,5304.911614,2899.971990,2292.657595,0.000000
10566,6829.639736,10725.924883,12182.127146,10036.286627,1373.863283,7984.219426,7309.393633,9040.255599,7150.222332,4406.261855,...,8955.752386,8203.888523,9056.805326,8670.159193,8394.178114,6483.795066,8188.145592,5783.205969,590.576383,2883.233979


In [123]:
ua_gc_odm

,2733,3280,3308,3883,5181,5280,5911,6315,6462,7000,...,10135,10200,10224,10241,10256,10328,10444,10453,10560,10565
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
1,2272.300389,1733.054532,5827.317668,6249.088465,9087.891376,1117.720698,1792.546491,654.146921,4835.894443,5255.419247,...,852.318141,1604.182003,1475.710189,1758.574475,1971.898411,2618.145058,5763.307033,3358.367410,8304.604476,6011.946881
4,5355.741805,2203.946211,7823.242767,8983.829009,12171.332792,4201.162114,4875.987908,3901.432542,7843.054291,8338.860663,...,4001.921375,4753.785238,4482.870037,4765.734324,4979.058260,5701.586475,8846.748450,6441.808826,11388.045892,9095.388297
5,1763.970750,2487.275347,5564.426244,5740.758826,8579.561737,609.391059,1284.216853,1261.782415,5150.066032,4747.089608,...,1606.538956,2195.556781,2229.931004,2512.795290,2726.119226,2109.815420,5254.977395,2850.037771,7796.274837,5503.617242
7,4046.910632,895.115037,6387.503549,7548.089791,10862.501619,2892.330941,3567.156735,2592.601369,6534.223118,7030.029490,...,2693.090202,3444.954065,3174.038864,3456.903150,3670.227086,4392.755301,7537.917276,5132.977653,10079.214719,7786.557124
8,3759.240518,1234.828634,5914.788022,7075.374265,10574.831506,2604.660828,3279.486621,2304.931256,6486.678777,6742.359377,...,2503.102475,3254.966338,3126.494523,3409.358809,3622.682746,4105.085188,7250.247163,4845.307539,9791.544606,7498.887010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10564,3127.243377,6563.352172,8479.730787,6333.890269,3688.347610,4281.823068,3606.997274,5337.859241,3375.311554,910.624762,...,5253.356027,4501.492164,4528.605300,4245.741013,4032.417077,2781.398707,4499.002383,2094.062759,2905.060710,612.403115
10565,3739.646492,7175.755287,9092.133902,6946.293384,3075.944495,4894.226183,4219.400389,5950.262356,3987.714669,1523.027877,...,5865.759142,5113.895279,5141.008415,4858.144128,4644.820192,3393.801822,5111.405498,2706.465874,2292.657595,0.000000
10566,6622.880471,10058.989266,11975.367881,9829.527362,1373.863283,7777.460161,7102.634368,8833.496334,6870.948648,4406.261855,...,8748.993121,7997.129258,8024.242393,7741.378107,7528.054171,6277.035801,7994.639477,5589.699853,590.576383,2883.233979


Set an index on the NN, reindex by an alphanumerically sorted list of IDs, then reset_index to return the NNs to the data frame

In [124]:
# allm = allm.set_index('NN').reindex(sorted_nicely(allm['NN'])).reset_index() # index, nodes are a bit messed up for the allm dataset
gcs = gcs.set_index('NN').reindex(sorted_nicely(gcs['NN'])).reset_index()

##### Georeference the origins files and spatial join admin files so they can be used for manipulation of gravity model data

In [125]:
current_orig_gc['geometry'] = current_orig_gc['geometry'].apply(wkt.loads)
current_orig_gc = gpd.GeoDataFrame(current_orig_gc,geometry='geometry')
current_orig_gc = gpd.sjoin(current_orig_gc,adm4[['geometry','ADM3_EN','ADM4_EN','ADM3_PCODE','ADM4_PCODE']],how='left',op='within')

C:\Users\wb467985\.conda\envs\geo\lib\site-packages\geopandas\tools\sjoin.py:61: UserWarning: CRS of frames being joined does not match!(None != epsg:4326)
  "(%s != %s)" % (left_df.crs, right_df.crs)


In [126]:
ua_orig_gc['geometry'] = ua_orig_gc['geometry'].apply(wkt.loads)
ua_orig_gc = gpd.GeoDataFrame(ua_orig_gc,geometry='geometry')
ua_orig_gc = gpd.sjoin(ua_orig_gc,adm4[['geometry','ADM3_EN','ADM4_EN','ADM3_PCODE','ADM4_PCODE']],how='left',op='within')

C:\Users\wb467985\.conda\envs\geo\lib\site-packages\geopandas\tools\sjoin.py:61: UserWarning: CRS of frames being joined does not match!(None != epsg:4326)
  "(%s != %s)" % (left_df.crs, right_df.crs)


### Gravity models

We implement gravity models for various scenarios and data points of interest. As per the charts, this is implemented in a for loop to allow quick production of bundles of data

The data points are from the economic census and were manually joined to the growth centers in QGIS.

In [127]:
import GOSTnets.calculate_od_raw as calcOD

Check out weighting options

In [128]:
# calcOD.calculate_gravity?? # for looking at options

#### Origins

Select the weights to use

In [130]:
gcs['good_jobs_4and5'] = gcs['good_jobs_fsize_4'] + gcs['good_jobs_fsize_5']
gcs['bad_jobs_4and5'] = gcs['bad_jobs_fsize_4'] + gcs['bad_jobs_fsize_5']
gcs['n_firms_4and5'] = gcs['number_firms_fsize_4'] + gcs['number_firms_fsize_5']
gcs['tw_ser_4and5'] = gcs['total_workers_ser_4'] + gcs['total_workers_ser_5']
gcs['tw_ind_4and5'] = gcs['total_workers_ind_4'] + gcs['total_workers_ind_5']
gcs['tw_tot'] = gcs['total_workers_tot_']
gcs['tw_tot_4and5'] = gcs['total_workers_fsize_4'] + gcs['total_workers_fsize_5']
gcs['n_firms_tot'] = gcs['number_firms_tot_']
gcs['no_weight'] = 1

In [131]:
gm_weights = ['good_jobs_tot_','bad_jobs_tot_','good_jobs_4and5','bad_jobs_4and5','n_firms_4and5','n_firms_tot','tw_tot','tw_tot_4and5','tw_ser_4and5','tw_ind_4and5','no_weight']

In [132]:
grav_models_dict = dict.fromkeys(gm_weights)

##### Current

Create a function to generate the gravity models and append them to a dictionary

In [135]:
def grav_loop(orig_file,od_matrix,oW_df,dW):
    orig_gravity = calcOD.calculate_gravity(np.array(od_matrix), oWeight=oW_df['Pop'], dWeight = dW)
    orig_gravity['NN'] = oW_df['O_ID']
    orig_gravity_gdf = pd.merge(orig_file, orig_gravity, how='left',left_on='O_ID',right_on='NN')

    return orig_gravity_gdf

Use the function to create gravity models for all the desired weights

In [136]:
for key, v in grav_models_dict.items():
    
    print(key)
    
    df = grav_loop(current_orig_gc,current_gc_odm,cur_orig_grp,gcs[key])
    
    gpkg_layername = 'gc_cur_' + key
    df.to_file(os.path.join('results\\spatial\\gravity_models.gpkg'),layer=gpkg_layername,driver="GPKG", ignore_errors=True)
    
    grav_models_dict[key] = df

good_jobs_tot_


CPLE_AppDefinedError: b'sqlite3_exec(CREATE VIRTUAL TABLE "rtree_gc_cur_good_jobs_tot__geom" USING rtree(id, minx, maxx, miny, maxy)) failed: no such module: rtree'

Exception ignored in: 'fiona._shim.gdal_flush_cache'
Traceback (most recent call last):
  File "fiona/_err.pyx", line 201, in fiona._err.GDALErrCtxManager.__exit__
fiona._err.CPLE_AppDefinedError: b'sqlite3_exec(CREATE VIRTUAL TABLE "rtree_gc_cur_good_jobs_tot__geom" USING rtree(id, minx, maxx, miny, maxy)) failed: no such module: rtree'


bad_jobs_tot_


CPLE_AppDefinedError: b'sqlite3_exec(CREATE VIRTUAL TABLE "rtree_gc_cur_bad_jobs_tot__geom" USING rtree(id, minx, maxx, miny, maxy)) failed: no such module: rtree'

...

CPLE_AppDefinedError: b'sqlite3_exec(CREATE VIRTUAL TABLE "rtree_gc_cur_tw_ind_4and5_geom" USING rtree(id, minx, maxx, miny, maxy)) failed: no such module: rtree'

Exception ignored in: 'fiona._shim.gdal_flush_cache'
Traceback (most recent call last):
  File "fiona/_err.pyx", line 201, in fiona._err.GDALErrCtxManager.__exit__
fiona._err.CPLE_AppDefinedError: b'sqlite3_exec(CREATE VIRTUAL TABLE "rtree_gc_cur_tw_ind_4and5_geom" USING rtree(id, minx, maxx, miny, maxy)) failed: no such module: rtree'


no_weight


CPLE_AppDefinedError: b'sqlite3_exec(CREATE VIRTUAL TABLE "rtree_gc_cur_no_weight_geom" USING rtree(id, minx, maxx, miny, maxy)) failed: no such module: rtree'

Exception ignored in: 'fiona._shim.gdal_flush_cache'
Traceback (most recent call last):
  File "fiona/_err.pyx", line 201, in fiona._err.GDALErrCtxManager.__exit__
fiona._err.CPLE_AppDefinedError: b'sqlite3_exec(CREATE VIRTUAL TABLE "rtree_gc_cur_no_weight_geom" USING rtree(id, minx, maxx, miny, maxy)) failed: no such module: rtree'


Aggregate the gravity models at the adm4 level, weighting by origin population

In [137]:
# iterate through the dict

adm4_grav = adm4

for key, layer in grav_models_dict.items():
    
    print(key)
    
    # weight index by population
    
    layer['adm4_pop'] = layer.groupby('ADM4_PCODE')['VALUE'].transform(np.sum)
    
    for col in layer.loc[:,'d_0.01':'d_1e-05']:
        layer[col] = (layer[col] * (layer['VALUE'] / layer['adm4_pop']))
    
#     [ col * (layer['VALUE'].astype(float) / layer['adm4_pop'].astype(float)) for col in layer.loc[:,'d_0.01':'d_1e-05']] # failed atempt at list comprehension
    
    # aggregate weighted indices by union
    
    union = layer.loc[:,'ADM4_PCODE':'d_1e-05'].groupby('ADM4_PCODE').apply(lambda x: x.sum())
    union.drop(columns='ADM4_PCODE',inplace=True)
    union = union.add_prefix(key + '_cur_')
    union.reset_index(inplace=True)
    union['ADM4_PCODE'] = union['ADM4_PCODE'].astype(str)

    # join to adm4 file

    adm4_grav = pd.merge(adm4_grav,union,how='left',on=['ADM4_PCODE'])  

good_jobs_tot_
bad_jobs_tot_
good_jobs_4and5
bad_jobs_4and5
n_firms_4and5
n_firms_tot
tw_tot
tw_tot_4and5
tw_ser_4and5
tw_ind_4and5
no_weight


##### All upgrades

Now repeat the process for the all upgraded roads data

In [138]:
for key, v in grav_models_dict.items():
    
    print(key)
    
    df = grav_loop(ua_orig_gc,ua_gc_odm,ua_orig_grp,gcs[key])
    
    gpkg_layername = 'gc_ua_' + key
    df.to_file(os.path.join('results\\spatial\\gravity_models.gpkg'),layer=gpkg_layername,driver="GPKG", ignore_errors=True)
    
    grav_models_dict[key] = df

good_jobs_tot_


CPLE_AppDefinedError: b'sqlite3_exec(CREATE VIRTUAL TABLE "rtree_gc_ua_good_jobs_tot__geom" USING rtree(id, minx, maxx, miny, maxy)) failed: no such module: rtree'

Exception ignored in: 'fiona._shim.gdal_flush_cache'
Traceback (most recent call last):
  File "fiona/_err.pyx", line 201, in fiona._err.GDALErrCtxManager.__exit__
fiona._err.CPLE_AppDefinedError: b'sqlite3_exec(CREATE VIRTUAL TABLE "rtree_gc_ua_good_jobs_tot__geom" USING rtree(id, minx, maxx, miny, maxy)) failed: no such module: rtree'


bad_jobs_tot_


CPLE_AppDefinedError: b'sqlite3_exec(CREATE VIRTUAL TABLE "rtree_gc_ua_bad_jobs_tot__geom" USING rtree(id, minx, maxx, miny, maxy)) failed: no such module: rtree'

...

CPLE_AppDefinedError: b'sqlite3_exec(CREATE VIRTUAL TABLE "rtree_gc_ua_tw_ind_4and5_geom" USING rtree(id, minx, maxx, miny, maxy)) failed: no such module: rtree'

Exception ignored in: 'fiona._shim.gdal_flush_cache'
Traceback (most recent call last):
  File "fiona/_err.pyx", line 201, in fiona._err.GDALErrCtxManager.__exit__
fiona._err.CPLE_AppDefinedError: b'sqlite3_exec(CREATE VIRTUAL TABLE "rtree_gc_ua_tw_ind_4and5_geom" USING rtree(id, minx, maxx, miny, maxy)) failed: no such module: rtree'


no_weight


CPLE_AppDefinedError: b'sqlite3_exec(CREATE VIRTUAL TABLE "rtree_gc_ua_no_weight_geom" USING rtree(id, minx, maxx, miny, maxy)) failed: no such module: rtree'

Exception ignored in: 'fiona._shim.gdal_flush_cache'
Traceback (most recent call last):
  File "fiona/_err.pyx", line 201, in fiona._err.GDALErrCtxManager.__exit__
fiona._err.CPLE_AppDefinedError: b'sqlite3_exec(CREATE VIRTUAL TABLE "rtree_gc_ua_no_weight_geom" USING rtree(id, minx, maxx, miny, maxy)) failed: no such module: rtree'


In [139]:
# iterate through the dict

# adm4_grav = adm4_grav

for key, layer in grav_models_dict.items():
    
    print(key)
    
    # weight index by population
    
    layer['adm4_pop'] = layer.groupby('ADM4_PCODE')['VALUE'].transform(np.sum)
    
    for col in layer.loc[:,'d_0.01':'d_1e-05']:
        layer[col] = (layer[col] * (layer['VALUE'] / layer['adm4_pop']))
    
#     [ col * (layer['VALUE'].astype(float) / layer['adm4_pop'].astype(float)) for col in layer.loc[:,'d_0.01':'d_1e-05']] # failed atempt at list comprehension
    
    # aggregate weighted indices by union
    
    union = layer.loc[:,'ADM4_PCODE':'d_1e-05'].groupby('ADM4_PCODE').apply(lambda x: x.sum())
    union.drop(columns='ADM4_PCODE',inplace=True)
    union = union.add_prefix(key + '_ua_')
    union.reset_index(inplace=True)
    union['ADM4_PCODE'] = union['ADM4_PCODE'].astype(str)

    # join to adm4 file

    adm4_grav = pd.merge(adm4_grav,union,how='left',on=['ADM4_PCODE'])  

good_jobs_tot_
bad_jobs_tot_
good_jobs_4and5
bad_jobs_4and5
n_firms_4and5
n_firms_tot
tw_tot
tw_tot_4and5
tw_ser_4and5
tw_ind_4and5
no_weight


Calculate changes in gravity resulting from UA scenario

In [140]:
adm4_grav['delta60_gj_tot'] = adm4_grav['good_jobs_tot__ua_d_0.0001925409'] - adm4_grav['good_jobs_tot__cur_d_0.0001925409']
adm4_grav['delta60_bj_tot'] = adm4_grav['bad_jobs_tot__ua_d_0.0001925409'] - adm4_grav['bad_jobs_tot__cur_d_0.0001925409']
adm4_grav['delta30_bj_tot'] = adm4_grav['bad_jobs_tot__ua_d_0.0003850818'] - adm4_grav['bad_jobs_tot__cur_d_0.0003850818']

adm4_grav['delta60_gj_4and5'] = adm4_grav['good_jobs_4and5_ua_d_0.0001925409'] - adm4_grav['good_jobs_4and5_cur_d_0.0001925409']
adm4_grav['delta60_bj_4and5'] = adm4_grav['bad_jobs_4and5_ua_d_0.0001925409'] - adm4_grav['bad_jobs_4and5_cur_d_0.0001925409']

adm4_grav['delta60_tw_tot'] = adm4_grav['tw_tot_ua_d_0.0001925409'] - adm4_grav['tw_tot_cur_d_0.0001925409']
adm4_grav['delta60_nf_tot'] = adm4_grav['n_firms_tot_ua_d_0.0001925409'] - adm4_grav['n_firms_tot_cur_d_0.0001925409']

adm4_grav['delta60_tw_4and5'] = adm4_grav['tw_tot_4and5_ua_d_0.0001925409'] - adm4_grav['tw_tot_4and5_cur_d_0.0001925409']
adm4_grav['delta60_nf_4and5'] = adm4_grav['n_firms_4and5_ua_d_0.0001925409'] - adm4_grav['n_firms_4and5_cur_d_0.0001925409']

adm4_grav['delta60_tw_ser4and5'] = adm4_grav['tw_ser_4and5_ua_d_0.0001925409'] - adm4_grav['tw_ser_4and5_cur_d_0.0001925409']
adm4_grav['delta60_tw_ind4and5'] = adm4_grav['tw_ind_4and5_ua_d_0.0001925409'] - adm4_grav['tw_ind_4and5_cur_d_0.0001925409']

adm4_grav['delta60_now'] = adm4_grav['no_weight_ua_d_0.0001925409'] - adm4_grav['no_weight_cur_d_0.0001925409']
adm4_grav['delta30_now'] = adm4_grav['no_weight_ua_d_0.0003850818'] - adm4_grav['no_weight_cur_d_0.0003850818']

In [142]:
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

Export the finished adm4 layer for data analysis and visualization

In [143]:
adm4_grav.to_file(os.path.join(res_pth,'spatial\\adm4_gravity_models.geojson'),driver="GeoJSON")

In [144]:
adm4_grav.to_file(os.path.join(res_pth,'spatial\\adm4_gravity_models.gpkg'),driver="GPKG", ignore_errors=True)

CPLE_AppDefinedError: b'sqlite3_exec(CREATE VIRTUAL TABLE "rtree_adm4_gravity_models_geom" USING rtree(id, minx, maxx, miny, maxy)) failed: no such module: rtree'

Exception ignored in: 'fiona._shim.gdal_flush_cache'
Traceback (most recent call last):
  File "fiona/_err.pyx", line 201, in fiona._err.GDALErrCtxManager.__exit__
fiona._err.CPLE_AppDefinedError: b'sqlite3_exec(CREATE VIRTUAL TABLE "rtree_adm4_gravity_models_geom" USING rtree(id, minx, maxx, miny, maxy)) failed: no such module: rtree'


#### Destinations

We invert the process here to show the relative gravity of each growth center (rather than the accessibility per adm4).

This allows us to display the importance of different GCs on maps of the accessibility outputs.

In [145]:
gm_weights = ['good_jobs_tot_','bad_jobs_tot_','good_jobs_4and5','bad_jobs_4and5','n_firms_4and5','n_firms_tot','tw_tot','tw_tot_4and5','tw_ser_4and5','tw_ind_4and5','no_weight']

In [146]:
grav_models_dict = dict.fromkeys(gm_weights)

Create a function to generate the gravity models and append them to a dictionary

In [147]:
def dest_grav_loop(orig_file,od_matrix,grav_param,oW_df,dW):
    dest_gravity = calcOD.calculate_gravity(np.array(od_matrix), oWeight=oW_df[grav_param], dWeight = dW['Pop'])
    dest_gravity['NN'] = oW_df['NN']
    dest_gravity_gdf = pd.merge(gcs, dest_gravity, how='left',on='NN')

    return dest_gravity_gdf

Run the function and export the results to a growth center gravity model geopackage, for later visualization in QGIS

In [148]:
for key, v in grav_models_dict.items():
    
    print(key)
    
    df = dest_grav_loop(current_orig_gc,current_gc_odm.transpose(),key,gcs,cur_orig_grp)
    
    gpkg_layername = 'gc_cur_' + key
    df.to_file(os.path.join('results\\spatial\\gravity_models_gcs.gpkg'),layer=gpkg_layername,driver="GPKG", ignore_errors=True)
    
    grav_models_dict[key] = df

good_jobs_tot_


CPLE_AppDefinedError: b'sqlite3_exec(CREATE VIRTUAL TABLE "rtree_gc_cur_good_jobs_tot__geom" USING rtree(id, minx, maxx, miny, maxy)) failed: no such module: rtree'

Exception ignored in: 'fiona._shim.gdal_flush_cache'
Traceback (most recent call last):
  File "fiona/_err.pyx", line 201, in fiona._err.GDALErrCtxManager.__exit__
fiona._err.CPLE_AppDefinedError: b'sqlite3_exec(CREATE VIRTUAL TABLE "rtree_gc_cur_good_jobs_tot__geom" USING rtree(id, minx, maxx, miny, maxy)) failed: no such module: rtree'


bad_jobs_tot_


CPLE_AppDefinedError: b'sqlite3_exec(CREATE VIRTUAL TABLE "rtree_gc_cur_bad_jobs_tot__geom" USING rtree(id, minx, maxx, miny, maxy)) failed: no such module: rtree'

...

CPLE_AppDefinedError: b'sqlite3_exec(CREATE VIRTUAL TABLE "rtree_gc_cur_tw_ind_4and5_geom" USING rtree(id, minx, maxx, miny, maxy)) failed: no such module: rtree'

Exception ignored in: 'fiona._shim.gdal_flush_cache'
Traceback (most recent call last):
  File "fiona/_err.pyx", line 201, in fiona._err.GDALErrCtxManager.__exit__
fiona._err.CPLE_AppDefinedError: b'sqlite3_exec(CREATE VIRTUAL TABLE "rtree_gc_cur_tw_ind_4and5_geom" USING rtree(id, minx, maxx, miny, maxy)) failed: no such module: rtree'


no_weight


CPLE_AppDefinedError: b'sqlite3_exec(CREATE VIRTUAL TABLE "rtree_gc_cur_no_weight_geom" USING rtree(id, minx, maxx, miny, maxy)) failed: no such module: rtree'

Exception ignored in: 'fiona._shim.gdal_flush_cache'
Traceback (most recent call last):
  File "fiona/_err.pyx", line 201, in fiona._err.GDALErrCtxManager.__exit__
fiona._err.CPLE_AppDefinedError: b'sqlite3_exec(CREATE VIRTUAL TABLE "rtree_gc_cur_no_weight_geom" USING rtree(id, minx, maxx, miny, maxy)) failed: no such module: rtree'


Repeat for upgraded all

In [149]:
for key, v in grav_models_dict.items():
    
    print(key)
    
    df = dest_grav_loop(ua_orig_gc,ua_gc_odm.transpose(),key,gcs,ua_orig_grp)
    
    gpkg_layername = 'gc_ua_' + key
    df.to_file(os.path.join('results\\spatial\\gravity_models_gcs.gpkg'),layer=gpkg_layername,driver="GPKG", ignore_errors=True)
    
    grav_models_dict[key] = df

good_jobs_tot_


CPLE_AppDefinedError: b'sqlite3_exec(CREATE VIRTUAL TABLE "rtree_gc_ua_good_jobs_tot__geom" USING rtree(id, minx, maxx, miny, maxy)) failed: no such module: rtree'

Exception ignored in: 'fiona._shim.gdal_flush_cache'
Traceback (most recent call last):
  File "fiona/_err.pyx", line 201, in fiona._err.GDALErrCtxManager.__exit__
fiona._err.CPLE_AppDefinedError: b'sqlite3_exec(CREATE VIRTUAL TABLE "rtree_gc_ua_good_jobs_tot__geom" USING rtree(id, minx, maxx, miny, maxy)) failed: no such module: rtree'


bad_jobs_tot_


CPLE_AppDefinedError: b'sqlite3_exec(CREATE VIRTUAL TABLE "rtree_gc_ua_bad_jobs_tot__geom" USING rtree(id, minx, maxx, miny, maxy)) failed: no such module: rtree'

...

CPLE_AppDefinedError: b'sqlite3_exec(CREATE VIRTUAL TABLE "rtree_gc_ua_tw_ind_4and5_geom" USING rtree(id, minx, maxx, miny, maxy)) failed: no such module: rtree'

Exception ignored in: 'fiona._shim.gdal_flush_cache'
Traceback (most recent call last):
  File "fiona/_err.pyx", line 201, in fiona._err.GDALErrCtxManager.__exit__
fiona._err.CPLE_AppDefinedError: b'sqlite3_exec(CREATE VIRTUAL TABLE "rtree_gc_ua_tw_ind_4and5_geom" USING rtree(id, minx, maxx, miny, maxy)) failed: no such module: rtree'


no_weight


CPLE_AppDefinedError: b'sqlite3_exec(CREATE VIRTUAL TABLE "rtree_gc_ua_no_weight_geom" USING rtree(id, minx, maxx, miny, maxy)) failed: no such module: rtree'

Exception ignored in: 'fiona._shim.gdal_flush_cache'
Traceback (most recent call last):
  File "fiona/_err.pyx", line 201, in fiona._err.GDALErrCtxManager.__exit__
fiona._err.CPLE_AppDefinedError: b'sqlite3_exec(CREATE VIRTUAL TABLE "rtree_gc_ua_no_weight_geom" USING rtree(id, minx, maxx, miny, maxy)) failed: no such module: rtree'
